# Model

In [1]:
import os
import sys

import pathlib

notebook_path = pathlib.Path(os.getcwd())
sys.path.append(str(notebook_path.parent))

In [2]:
import numpy as np
import pandas as pd

import pickle

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder
)
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV,
    train_test_split
) 
from category_encoders import TargetEncoder

from IPython.display import HTML

from database import engine

## Loading data

- We only load records that belong to the `view` category, because every like starts with view, and we have a `target` column that marks those views that lead to likes.

In [3]:
user_data = pd.read_sql(
    "SELECT * FROM public.user_data;",
    con = engine,
    index_col = "user_id"
)
post_data = pd.read_sql(
    "SELECT * FROM public.post_text_df;",
    con = engine,
    index_col = "post_id"
)
feed_data = pd.read_sql(
    "SELECT * FROM public.feed_data LIMIT 100;",
    con = engine
)

Sometimes this frame loads too slowly, so I use a temporary local file that stores the data locally.

In [4]:
joined_data = pd.read_parquet(
    "preloaded_joined_data.parquet"
).sample(100000, random_state=10)

In [5]:
df_show = {
    "Users data" : user_data,
    "Post data" : post_data,
    "Feed data" : feed_data,
    "Joined data" : joined_data
}

for title, df in df_show.items():
    display(HTML(f"<h3>{title}</h3>"))
    display(df.head())

,gender,age,country,city,exp_group,os,source
user_id,,,,,,,
200,1,34,Russia,Degtyarsk,3,Android,ads
201,0,37,Russia,Abakan,0,Android,ads
202,1,17,Russia,Smolensk,4,Android,ads
203,0,18,Russia,Moscow,1,iOS,ads
204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


,text,topic
post_id,,
1,UK economy facing major risks\n\nThe UK manufa...,business
2,Aids and climate top Davos agenda\n\nClimate c...,business
3,Asian quake hits European shares\n\nShares in ...,business
4,India power shares jump on debut\n\nShares in ...,business
5,Lacroix label bought by US firm\n\nLuxury good...,business


,timestamp,user_id,post_id,action,target
0,2021-11-29 20:24:55,134897,3109,view,0
1,2021-11-29 20:26:23,134897,4007,view,0
2,2021-11-29 20:27:21,134897,6055,view,0
3,2021-11-29 20:28:22,134897,4869,view,0
4,2021-11-29 20:30:34,134897,5166,view,0


,user_id,timestamp,target,gender,age,country,city,exp_group,os,source,text,topic
71213,160438,2021-11-25 17:17:56,0,1,23,Russia,Kaluga,3,iOS,organic,Irish company hit by Iraqi report\n\nShares in...,business
381916,104477,2021-10-21 07:04:56,0,0,23,Russia,Bol’shaya Yelkhovka,2,iOS,ads,this is the 4th movie in the Karate Kid series...,movie
423558,161074,2021-11-16 15:14:57,0,1,29,Russia,Kirishi,2,Android,organic,"The whole town of Blackstone is afraid, becaus...",movie
62340,113212,2021-11-26 11:27:00,0,1,18,Russia,Kazan,1,Android,organic,"I grew up watching, and loving this cartoon ev...",movie
466131,19018,2021-12-21 19:31:58,0,0,20,Russia,Perm,1,iOS,ads,The Movie is okay. Meaning that I dont regret ...,movie


## Model pipeline

In [6]:
X = joined_data.drop("user_id", axis = 1).copy()

y = X["target"]
X.drop("target", axis = 1, inplace = True)

X["month"] = X["timestamp"].dt.month
X["year"] = X["timestamp"].dt.year
X["hour"] = X["timestamp"].dt.hour
X.drop("timestamp", axis = 1, inplace = True)

X['gender'] = X['gender'].astype("O")

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size = 0.8, random_state = 1
)

In [8]:
numeric_columns = list(X_train.select_dtypes("number").columns)
categorical_columns = [
    'gender', 'country', 'city', 'os', 'source', 'topic'
]
cat_nunique = X_train[categorical_columns].nunique()
MTE_columns = cat_nunique.index[cat_nunique > 5].to_list()
OHE_columns = list(set(categorical_columns) - set(MTE_columns))
text_columns = 'text'

In [9]:
data_transformer = ColumnTransformer([
    ("stand_scaler", StandardScaler(), numeric_columns),
    (
        "one_hot_encod", 
        OneHotEncoder(
            categories = list(X_train[OHE_columns].apply(
                lambda col: list(col.unique()),
                result_type="reduce"
            ))
        ), 
        OHE_columns
    ),
    (
        "mean_target_encoder", 
        TargetEncoder(min_samples_leaf = 0.5, smoothing = 0.5), 
        MTE_columns
    ),
    ("tf_idf_vector", TfidfVectorizer(max_features = 10), text_columns)
])

In [10]:
pipeline = Pipeline([
    ("transfmer", data_transformer),
    ("model", GradientBoostingClassifier())
])

## Model selection

In [11]:
param_grid = {
    "model__learning_rate" : [1.5, 0.2, 0.3],
    "model__max_depth" : [3,4,5],
    "model__n_estimators" : [50, 60, 70],
    "transfmer__tf_idf_vector__max_features" : [5, 10, 15, 20]
}

grid_search_result = GridSearchCV(
    estimator = pipeline,
    param_grid = param_grid,
    verbose = 2,
    scoring = "roc_auc",
    return_train_score = True
).fit(X_train, y_train)
pickle.dump(grid_search_result, open("gs_results.pck", "wb"))

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END model__learning_rate=1.5, model__max_depth=3, model__n_estimators=50, transfmer__tf_idf_vector__max_features=5; total time=  13.7s
[CV] END model__learning_rate=1.5, model__max_depth=3, model__n_estimators=50, transfmer__tf_idf_vector__max_features=5; total time=  14.1s
[CV] END model__learning_rate=1.5, model__max_depth=3, model__n_estimators=50, transfmer__tf_idf_vector__max_features=5; total time=  14.2s
[CV] END model__learning_rate=1.5, model__max_depth=3, model__n_estimators=50, transfmer__tf_idf_vector__max_features=5; total time=  14.1s
[CV] END model__learning_rate=1.5, model__max_depth=3, model__n_estimators=50, transfmer__tf_idf_vector__max_features=5; total time=  14.4s
[CV] END model__learning_rate=1.5, model__max_depth=3, model__n_estimators=50, transfmer__tf_idf_vector__max_features=10; total time=  15.9s
[CV] END model__learning_rate=1.5, model__max_depth=3, model__n_estimators=50, transfmer__tf_idf

In [12]:
gs_results_frame = pd.DataFrame(grid_search_result.cv_results_["params"])
gs_results_frame["mean_train_score"] = grid_search_result.cv_results_["mean_test_score"]
gs_results_frame["mean_test_score"] = grid_search_result.cv_results_["mean_train_score"]
gs_results_frame.sort_values("mean_train_score", ascending = False)

,model__learning_rate,model__max_depth,model__n_estimators,transfmer__tf_idf_vector__max_features,mean_train_score,mean_test_score
59,0.2,4,70,20,0.642575,0.736534
55,0.2,4,60,20,0.642094,0.730729
83,0.3,3,70,20,0.641134,0.719360
79,0.3,3,60,20,0.641129,0.715228
82,0.3,3,70,15,0.640824,0.719025
...,...,...,...,...,...,...
34,1.5,5,70,15,0.596681,0.784727
31,1.5,5,60,20,0.594873,0.779732
35,1.5,5,70,20,0.594765,0.786577
33,1.5,5,70,10,0.593987,0.773435


In [13]:
grid_search_result = pickle.load(open("gs_results.pck", "rb"))

best_model = pipeline.set_params(
    **grid_search_result.best_params_
).fit(X_train, y_train)

file_name = "model.pck"
pickle.dump(best_model, open(file_name, "wb"))

## Hitrage@5 estimation

In [14]:
preds_test = pd.Series(
    best_model.predict_proba(X_test)[:,1],
    index = y_test.index
)

data_for_estimation = pd.DataFrame({
    "preds" : preds_test,
    "y" : y_test,
    "user_id" : joined_data.loc[y_test.index, "user_id"]
})

hit_rages = (
    data_for_estimation.
    groupby("user_id").
    apply(lambda x: x.nlargest(5, "preds")["y"].any() if len(x) >= 5 else None)
).dropna().astype("int32")

print("current hitrage@5", hit_rages.mean())

current hitrage@5 0.5002478929102627
